<a href="https://colab.research.google.com/github/carlibeisel/mason_water_budget/blob/main/reach_drainflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reach Drainflow**

By Carli Beisel

Created: August 2024

Purpose: Compile and clean drainflow data for each reach.



**Reach**

There are 6 reaches total in Mason Drainage. 4 on Mason Creek, 1 on Solomon Drain, and 1 on Purdum Drain.

*Qin* is the upstream gauge on the Reach, and *Qout* is the downstream gauge.

In [1]:
## --------------- ##
## IMPORT PACKAGES ##
## --------------- ##

import pandas as pd # to work with dataframe
import os # for file paths
import glob # read in a folder of csv
import numpy as np # basic statistics

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## --------------- ##
##  LOAD DATASETS  ##
## --------------- ##

drainflows = glob.glob ('/content/drive/MyDrive/Data/Mason/drainflow/ethos_gauges/*.csv')

In [4]:
file = pd.read_csv('/content/drive/MyDrive/Data/Mason/drainflow/ethos_gauges/Purdum at Sweetwater.csv')

In [ ]:
## --------------- ##
##    Clean Data   ##
## --------------- ##

# remove NA and 0

In [10]:
## ------------------------------------- ##
##   Average daily flow for each gauge   ##
## ------------------------------------- ##

average_streamflows = pd.DataFrame()

for file in drainflows:
    df = pd.read_csv(file)
    df['Time'] = pd.to_datetime(df['Time'])
    df['Date'] = df['Time'].dt.date
    gauge_name = os.path.basename(file).replace('.csv', '') # extract the gauge name from the file
    daily_avg = df.groupby('Date')['Flow'].mean().reset_index() #g roup by date and compute flows
    daily_avg['Gauge'] = gauge_name # add gauge name as new column
    average_streamflows = pd.concat([average_streamflows, daily_avg])

average_streamflows.to_csv('/content/drive/MyDrive/Data/Mason/drainflow/ethos_gauges/average_drainflows.csv', index=False)


In [15]:
df = pd.read_csv('/content/drive/MyDrive/Data/Mason/drainflow/ethos_gauges/average_drainflows.csv')

In [14]:
## -------------------------------- ##
##   Compile data in to each Reach  ##
## -------------------------------- ##

# Reach 1 (Mason Creek at Phyllis --> Mason Creek at Madison)
#reach1_qin =
#reach1_qout =

# Reach 2 (Mason Creek at Madison --> Mason Creek at Ustick and Northside)
#reach2_qin =
#reach2_qout =

# Reach 3 (Mason Creek at Ustick and Northside --> Mason Creek at Marble Front Road)
#reach3_qin =
#reach3_qout =

# Reach 4 (Mason Creek at Marble Front Road --> Mason Creek USGS Gauge)
#reach4_qin =
#reach4_qout =


# Purdum Reach (Purdum at Phyllis --> Purdum at Sweetwater)
purdum_qin = df['Purdum at Phyllis']
purdum_qout = df['Purdum at Sweetwater']

# Solomon Reach (Solomon Drain at Highline Canal --> Solomon Drain at Marble Front Road)
#solomon_qin = df['Solomon Drain at Highline Canal']
#solomon_qout = df['Solomon Drain at Marble Front Road']

KeyError: 'Purdum at Phyllis'

In [ ]:
## -------------------------------- ##
##      Export Final Datasets       ##
## -------------------------------- ##

# Indiviudal Reach Dataset

output_directory = '/content/drive/MyDrive/Data/Mason/drainflow/reach_drainflow/'

